In [1]:
from pyomo.environ import *
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from dash import Dash, dcc, html
import webbrowser
from threading import Timer
import dash_bootstrap_components as dbc
import plotly.io as pio
from dash.dependencies import Input, Output
import json


In [2]:
# Read price data from Excel
price_data = pd.read_excel('data/price_data_arb_reg_separate_res.xlsx', sheet_name= None, index_col=0)
schedule_data = pd.read_excel('data/schedule_data.xlsx', sheet_name= None, index_col=0)

data = price_data

# Iteratively merge all dataframes on 'Timestamp'
price_df = pd.DataFrame(index =  data[list(data.keys())[0]].index)

for key in data:
    price_df = pd.concat([price_df, data[key]], axis = 1)

λ_min = schedule_data['soc_limit']['min']
λ_max = schedule_data['soc_limit']['max']
theta = schedule_data['availability']

z = {}
s = {}
z['arb'], z['reg'], z['pres'], z['cres'], z['opp'] = schedule_data['basis'].value.values
s['reg'] = schedule_data['basis'].loc['regulation', 'sym']

In [3]:
# Parameters
Seff = 1
Ceff = 1
Deff = 1
cap_power = 1  # Example value
cap_energy = 4  # Example value

λ_0 = 0.5
initial_soc = λ_0 * cap_energy

kappa = 0.95     # regulation performance score
beta = 0.15       # activated portion for reserve


cycle_life = 7300000
calendar_life = 20
dt = 1
total_time_period = len(price_df)
model_time_period = 24    # periods
num_slices = int(total_time_period // model_time_period)     # number of time slices to model

η = cycle_life/calendar_life/num_slices                 # number of cycle per time slice

# Model

In [4]:
def optimize_revenue(initial_soc, price_vector, current_period, first_period, last_period):

    # Define the model
    model = ConcreteModel()

    # Price Data for the day
    total_time_period = len(price_vector)

    T = range(1, total_time_period + 1)
    SOC_T = range(0, total_time_period + 1)

    p_arb = {t: price_vector['arb_energy_price'].values[t-1] for t in T}
    p_pres = {t: price_vector['pres_capacity_price'].values[t-1] for t in T}
    p_cres = {t: price_vector['cres_capacity_price'].values[t-1] for t in T}

    # p_reg = {t: price_vector['reg_capacity_price'].values[t-1] for t in T}
    p_reg_down = {t: price_vector['reg_down_price'].values[t-1] for t in T}
    p_reg_up = {t: price_vector['reg_up_price'].values[t-1] for t in T}

    start_soc = λ_0 * cap_energy

    # Variables
    model.SOC = Var(SOC_T, within=NonNegativeReals, bounds = (0, cap_energy))
 
    model.C_arb = Var(T, within=NonNegativeReals, bounds = (0, 999))
    model.D_arb = Var(T, within=NonNegativeReals, bounds = (0, 999))
    model.C_reg = Var(T, within=NonNegativeReals, bounds = (0, 999))
    model.D_reg = Var(T, within=NonNegativeReals, bounds = (0, 999))
    model.D_pres = Var(T, within=NonNegativeReals, bounds = (0, 999))
    model.D_cres = Var(T, within=NonNegativeReals, bounds = (0, 999))

    model.y_ch = Var(T, within=Binary)
    model.y_minus = Var(T, within=Binary)

    model.R_arb = Var(T, within=Reals)
    model.R_reg = Var(T, within=Reals)
    model.R_pres = Var(T, within=Reals)
    model.R_cres = Var(T, within=Reals)

    # Objective function
    model.obj = Objective(
        expr= sum(model.R_arb[t] + model.R_reg[t] + model.R_pres[t] + model.R_cres[t] for t in T),        
        sense=maximize
    )

    def revenue_arbitrage_rule(model, t):
        return model.R_arb[t] == p_arb[t]  * (model.D_arb[t] - model.C_arb[t]) * dt
    model.revenue_arbitrage = Constraint(T, rule=revenue_arbitrage_rule)

    def revenue_regulation_rule(model, t):
        return model.R_reg[t] == kappa * (p_reg_down[t] * model.C_reg[t] + p_reg_up[t] * model.D_reg[t]) * dt
    model.revenue_regulation = Constraint(T, rule=revenue_regulation_rule)

    def revenue_primary_reserve_rule(model, t):
        return model.R_pres[t] == p_pres[t]  * model.D_pres[t] * dt
    model.revenue_preserve = Constraint(T, rule=revenue_primary_reserve_rule)

    def revenue_contingency_reserve_rule(model, t):
        return model.R_cres[t] == p_cres[t]  * model.D_cres[t] * dt
    model.revenue_creserve = Constraint(T, rule=revenue_contingency_reserve_rule)

    # Arbitrage Constraints

    def c_limit_constraint(model, t):
        return model.C_arb[t] <= cap_power * model.y_ch[t]
    model.C_limit = Constraint(T, rule=c_limit_constraint)

    def d_limit_constraint(model, t):
        return model.D_arb[t]  <= cap_power * (1 - model.y_ch[t])
    model.D_limit = Constraint(T, rule=d_limit_constraint)

    def soc_constraints(model, t):
        if t == 0:
            return model.SOC[t] == initial_soc
        else:
            # return model.SOC[t] == model.SOC[t-1] * Seff + ((model.C_arb[t] + 0.15 * model.C_reg[t]) * Ceff - (model.D_arb[t] + 0.15 * model.D_reg[t])/Deff) * dt
            return model.SOC[t] == model.SOC[t-1] * Seff + (model.C_arb[t]  * Ceff - model.D_arb[t]/Deff) * dt
    model.SOC_constraints = Constraint(SOC_T, rule=soc_constraints)

    def soc_lower_limit_constraint(model, t):
        return model.SOC[t] >= cap_energy * λ_min[t]
    model.SOC_lower_limit = Constraint(T, rule=soc_lower_limit_constraint)

    def soc_upper_limit_constraint(model, t):
        return model.SOC[t] <= cap_energy * λ_max[t]
    model.SOC_upper_limit = Constraint(T, rule=soc_upper_limit_constraint)

    if current_period == last_period:
        def soc_final_state_constraint(model):
            end_t = SOC_T[-1]
            return model.SOC[end_t] == start_soc 
        model.SOC_final = Constraint(rule=soc_final_state_constraint)


    # Frequency and reserve constraints
    def total_charge_rule(model, t):
        return model.C_reg[t] - (model.D_arb[t] - model.C_arb[t])  <= cap_power 
    model.total_charge = Constraint(T, rule=total_charge_rule)

    def total_discharge_rule(model, t):
        return model.D_reg[t] + (model.D_arb[t] - model.C_arb[t]) +  model.D_pres[t] +  model.D_cres[t] <= cap_power 
    model.total_discharge = Constraint(T, rule=total_discharge_rule)

    def soc_reg_down_rule(model, t):
        return model.SOC[t] + beta * model.C_reg[t] * Ceff * dt <= cap_energy
    model.soc_reg_down = Constraint(T, rule=soc_reg_down_rule)

    def soc_reg_up_rule(model, t):
        return model.SOC[t] >= (beta * model.D_reg[t] + model.D_pres[t] + model.D_cres[t])/Deff * dt
    model.soc_reg_up = Constraint(T, rule=soc_reg_up_rule)

    if s['reg'] == 1:
        def reg_constraint_rule(model, t):
            return model.C_reg[t] == model.D_reg[t]
        model.reg_up_down = Constraint(T, rule=reg_constraint_rule)


    if z['opp'] == 0:
        def total_charge_no_opp_rule(model, t):
            return model.C_reg[t]  + model.C_arb[t]  <= cap_power * model.y_minus[t]
        model.total_charge_nopp = Constraint(T, rule=total_charge_no_opp_rule)

        def total_discharge_no_opp_rrule(model, t):
            return model.D_reg[t] + model.D_arb[t] +  model.D_pres[t] +  model.D_cres[t] <= cap_power * (1 - model.y_minus[t])
        model.total_discharge_nopp = Constraint(T, rule=total_discharge_no_opp_rrule)


    # Cycle limits

    def energy_cap_C_constraint(model):
        return sum(model.C_arb[t] +  model.C_reg[t]*beta for t in T)*dt <= η * cap_energy
        # return sum(model.C_arb[t] for t in T)*dt <= η * cap_energy
    model.energy_cap_C = Constraint(rule=energy_cap_C_constraint)

    def energy_cap_D_constraint(model):
        return sum(model.D_arb[t] + model.D_reg[t]*beta for t in T)*dt <= η * cap_energy
        # return sum(model.D_arb[t] for t in T)*dt <= η * cap_energy
    model.energy_cap_D = Constraint(rule=energy_cap_D_constraint)


    # Solve the model
    solver = SolverFactory('glpk')
    solver.solve(model, tee=True)

    # Extract results for charging, discharging, and SOC
    charging_schedule = [model.C_arb[t].value for t in T]  # Charging as negative
    discharging_schedule = [model.D_arb[t].value for t in T]  # Discharging as positive
    reg_down_schedule = [model.C_reg[t].value for t in T]  # Charging as negative
    reg_up_schedule = [model.D_reg[t].value for t in T]  # Discharging as positive
    pres_schedule = [model.D_pres[t].value for t in T]  # Charging as negative
    cres_schedule = [model.D_cres[t].value for t in T]  # Discharging as positive

    soc_schedule = [model.SOC[t].value for t in SOC_T]  # State of Charge

    rev_arbitrage =  sum(model.R_arb[t] for t in T)() 
    rev_regulation = sum(model.R_reg[t] for t in T)() 
    rev_cont_reserve = sum(model.R_cres[t] for t in T)() 

    # Return the results and final SOC
    return model.obj(), [charging_schedule, discharging_schedule, soc_schedule, reg_down_schedule, reg_up_schedule, pres_schedule, cres_schedule], [rev_arbitrage, rev_regulation, rev_cont_reserve]

# Run

In [5]:
# Lists to store results
total_revenue = 0
arbitrage = 0
regulation = 0
creserve = 0

all_charging_schedules = []
all_discharging_schedules = []
all_reg_down_schedules = []
all_reg_up_schedules = []
all_pres_schedules = []
all_cres_schedules = []
all_soc_schedules = []

first_p = 0
last_p = num_slices - 1

# Run the optimization for each time period
for p in range(num_slices):
    print("watch", p)
    periodic_price = price_df[p*model_time_period:(p+1)*model_time_period]
    revenue, [charging_schedule, discharging_schedule, soc_schedule, reg_down_schedule, reg_up_schedule, pres_schedule, cres_schedule], [rev_arbitrage, rev_regulation, rev_cont_reserve] = optimize_revenue(initial_soc, periodic_price, p, first_p, last_p)
    final_soc = soc_schedule[-1]
    
    # Store the results
    total_revenue += revenue
    arbitrage += rev_arbitrage
    regulation += rev_regulation
    creserve += rev_cont_reserve

    all_charging_schedules.extend(charging_schedule)
    all_discharging_schedules.extend(discharging_schedule)
    all_reg_down_schedules.extend(reg_down_schedule)
    all_reg_up_schedules.extend(reg_up_schedule)
    all_pres_schedules.extend(pres_schedule)
    all_cres_schedules.extend(cres_schedule)
    all_soc_schedules.extend(soc_schedule[:-1])

    # Update the initial SOC for the next day
    initial_soc = final_soc

print(total_revenue)

watch 0
GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write C:\Users\angelmah\AppData\Local\Temp\tmppdapwe60.glpk.raw --wglp
 C:\Users\angelmah\AppData\Local\Temp\tmpu7y1kl5b.glpk.glp --cpxlp C:\Users\angelmah\AppData\Local\Temp\tmp4dscjdp8.pyomo.lp
Reading problem data from 'C:\Users\angelmah\AppData\Local\Temp\tmp4dscjdp8.pyomo.lp'...
C:\Users\angelmah\AppData\Local\Temp\tmp4dscjdp8.pyomo.lp:2205: warning: lower bound of variable 'x291' redefined
C:\Users\angelmah\AppData\Local\Temp\tmp4dscjdp8.pyomo.lp:2205: warning: upper bound of variable 'x291' redefined
315 rows, 289 columns, 865 non-zeros
24 integer variables, all of which are binary
2229 lines were read
Writing problem data to 'C:\Users\angelmah\AppData\Local\Temp\tmpu7y1kl5b.glpk.glp'...
2028 lines were written
GLPK Integer Optimizer, v4.65
315 rows, 289 columns, 865 non-zeros
24 integer variables, all of which are binary
Preprocessing...
168 rows, 192 columns, 527 non-zeros
24 integer varia

In [6]:
arbitrage

57881.61740209961

In [7]:
regulation

196948.71916497534

In [8]:
creserve

30280.368509479555

In [9]:
# Create a DataFrame for the results
data = {
    't': range(1, len(all_charging_schedules) + 1),
    'charge': all_charging_schedules,
    'discharge': all_discharging_schedules,
    'reg_down': all_reg_down_schedules,
    'reg_up': all_reg_up_schedules,
    'pres': all_pres_schedules,
    'cres': all_cres_schedules,
    'soc': all_soc_schedules
}

result_df = pd.DataFrame(data)

result_df = result_df.set_index(['t'])

result_df['net_discharge'] = result_df['discharge'] - result_df['charge']
result_df['soc_percent'] = result_df['soc'] /cap_energy

result_df

,charge,discharge,reg_down,reg_up,pres,cres,soc,net_discharge,soc_percent
t,,,,,,,,,
1,0.0,1.000000e+00,2.000000,0.000000e+00,0.0,0.0,2.000000,1.000000,0.500000
2,1.0,-2.220446e-16,0.000000,2.000000e+00,0.0,0.0,1.000000,-1.000000,0.250000
3,1.0,0.000000e+00,0.000000,2.000000e+00,0.0,0.0,2.000000,-1.000000,0.500000
4,0.0,1.000000e+00,2.000000,0.000000e+00,0.0,0.0,3.000000,1.000000,0.750000
5,0.0,1.000000e+00,2.000000,-7.401487e-16,0.0,0.0,2.000000,1.000000,0.500000
...,...,...,...,...,...,...,...,...,...
8756,1.0,0.000000e+00,0.000000,2.000000e+00,0.0,0.0,0.150824,-1.000000,0.037706
8757,1.0,0.000000e+00,0.000000,2.000000e+00,0.0,0.0,1.150824,-1.000000,0.287706
8758,0.0,1.508243e-01,1.150824,8.491757e-01,0.0,0.0,2.150824,0.150824,0.537706


In [10]:
price_df.columns

Index(['arb_energy_price', 'reg_down_price', 'reg_up_price',
       'pres_capacity_price', 'pres_energy_price', 'cres_capacity_price',
       'cres_energy_price'],
      dtype='object')

In [22]:
import pandas as pd
import numpy as np
import json

# Sample data creation for demonstration
np.random.seed(42)
date_range = pd.date_range(start='2023-01-01', periods=100, freq='H')
price_df = pd.DataFrame({
    'date': date_range,
    'arb_energy_price': np.random.rand(len(date_range)) * 10,
    'reg_up_price': np.random.rand(len(date_range)) * 10,
    'reg_down_price': np.random.rand(len(date_range)) * 10,
    'cres_capacity_price': np.random.rand(len(date_range)) * 10,
})

# Convert data to JSON format
price_data_json = price_df.to_json(orient='split', date_format='iso')


In [25]:
html_template = f"""
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Interactive Plotly Chart with Multiplier</title>
    <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
</head>
<body>
    <h1>Charging and Discharging Schedule</h1>
    <div>
        <label for="multiplier">Multiplier:</label>
        <input type="range" id="multiplier" name="multiplier" min="0.1" max="5.0" step="0.1" value="1.0" oninput="updatePlot(this.value)">
        <span id="multiplier-value">1.0</span>
    </div>
    <div id="chart"></div>

    <script>
        var priceData = {price_data_json};

        function createPlot(multiplier) {{
            var dateRange = priceData.index.map(date => new Date(date));
            var arb_energy = priceData.data.map(row => row[1] * (multiplier^0.8));
            var reg_up = priceData.data.map(row => row[2] * multiplier^0.9);
            var reg_down = priceData.data.map(row => row[3] * multiplier^1.1);
            var cres_capacity = priceData.data.map(row => row[4] * multiplier^0.5);

            var data = [
                {{ x: dateRange, y: arb_energy, mode: 'lines', name: 'Energy', line: {{ color: '#4e79a7' }} }},
                {{ x: dateRange, y: reg_up, mode: 'lines', name: 'Regulation up', line: {{ color: '#e15759' }} }},
                {{ x: dateRange, y: reg_down, mode: 'lines', name: 'Regulation down', line: {{ color: '#76b7b2' }} }},
                {{ x: dateRange, y: cres_capacity, mode: 'lines', name: 'Reserve', line: {{ color: '#59a14f' }} }}
            ];

            var layout = {{
                title: 'Charging and Discharging Schedule',
                xaxis: {{ title: 'Time' }},
                yaxis: {{ title: 'Power (MW)' }},
                legend: {{
                    orientation: 'h',
                    yanchor: 'bottom',
                    y: 1.1,
                    xanchor: 'center',
                    x: 0.5
                }},
                autosize: true,
                margin: {{ l: 40, r: 40, t: 120, b: 40 }}
            }};

            Plotly.newPlot('chart', data, layout);
        }}

        function updatePlot(value) {{
            document.getElementById('multiplier-value').innerText = value;
            createPlot(parseFloat(value));
        }}

        // Initialize plot
        createPlot(1.0);
    </script>
</body>
</html>
"""

# Save the HTML to a file
file_path = "interactive_plot.html"
with open(file_path, "w", encoding="utf-8") as file:
    file.write(html_template)

file_path


'interactive_plot.html'

In [12]:
# Create interactive plot for charging and discharging
fig_prices = go.Figure()
fig_prices.add_trace(go.Scatter(x=price_df.index, y=price_df['arb_energy_price'], mode='lines', name='Energy', line=dict(color='#4e79a7')))
fig_prices.add_trace(go.Scatter(x=price_df.index, y=price_df['reg_up_price'], mode='lines', name='Regulation up', line=dict(color='#e15759')))
fig_prices.add_trace(go.Scatter(x=price_df.index, y=price_df['reg_down_price'], mode='lines', name='Regulation down', line=dict(color='#76b7b2')))
fig_prices.add_trace(go.Scatter(x=price_df.index, y=price_df['cres_capacity_price'], mode='lines', name='Reserve', line=dict(color='#59a14f')))

fig_prices.update_layout(
    title='Charging and Discharging Schedule',
    xaxis_title='Time',
    yaxis_title='Power (MW)',
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.1,
        xanchor="center",
        x=0.5
    ),
    autosize=True,
    margin=dict(l=40, r=40, t=120, b=40),  # Increase the top margin to make space for the legend
)

fig_prices.show()

fig_charge_discharge = go.Figure()
fig_charge_discharge.add_trace(go.Scatter(x=result_df.index, y=result_df['net_discharge'], mode='lines', name='Net battery power', line=dict(color='#4e79a7')))
fig_charge_discharge.add_trace(go.Scatter(x=result_df.index, y=result_df['reg_up'], mode='lines', name='Regulation up', line=dict(color='#e15759')))
fig_charge_discharge.add_trace(go.Scatter(x=result_df.index, y=result_df['reg_down'], mode='lines', name='Regulation down', line=dict(color='#76b7b2')))
fig_charge_discharge.add_trace(go.Scatter(x=result_df.index, y=result_df['cres'], mode='lines', name='Reserve', line=dict(color='#59a14f ')))

fig_charge_discharge.update_layout(
    title='Charging and Discharging Schedule',
    xaxis_title='Time',
    yaxis_title='Power (MW)',
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.1,
        xanchor="center",
        x=0.5
    ),
    autosize=True,
    margin=dict(l=40, r=40, t=120, b=40),  # Increase the top margin to make space for the legend
)

fig_charge_discharge.show()


# Create interactive plot for charging and discharging
fig_soc = go.Figure()
fig_soc.add_trace(go.Scatter(x=result_df.index, y=result_df['soc_percent']*100, mode='lines', name='net', line=dict(color='#4e79a7')))
fig_soc.update_layout(
    title='State of Charge',
    xaxis_title='Time',
    yaxis_title='SOC (%)',
    # xaxis=dict(
    #     tickmode='linear',
    #     dtick=86400000.0 * 30,  # 86400000.0 milliseconds in a day * 30 days
    # ),
    legend=dict(x=1.05, y=1),  # Position the legend outside the plot
    autosize=True,
    margin=dict(l=40, r=40, t=40, b=40),
)
fig_soc.show()

In [13]:
# Create the combined layout
combined_html = f"""
<html>
    <head>
        <title>BESS Revenue Simulation Dashboard</title>
        <style>
            .responsive-plot {{
                width: 80%;
                max-width: 3600px;
                height: 80%;
                max-height: 600px;
                margin: auto;
            }}
        </style>
        <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>

    </head>
    <body>
        <h1>BESS Revenue Simulation Dashboard</h1>

        <div>
            <h3>Prices</h3>
            <div id="plot1" class="responsive-plot">
                {pio.to_html(fig_prices, include_plotlyjs=False, full_html=False)}
            </div>
        </div>


        <div>
            <h3>Charging and Discharging Schedules</h3>
            <div id="plot1" class="responsive-plot">
                {pio.to_html(fig_charge_discharge, include_plotlyjs=False, full_html=False)}
            </div>
        </div>

        <div>
            <h3>State of Charge Over Time</h3>
            <div id="plot2" class="responsive-plot">
                {pio.to_html(fig_soc, include_plotlyjs=False, full_html=False)}
            </div>
        </div>
    </body>
</html>
"""

# Save the combined HTML to a file
with open("bess_dashboard.html", "w") as f:
    f.write(combined_html)